In [1]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [2]:
import tkinter as tk
from tkinter import messagebox
import sqlite3
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

# Function to calculate BMI
def calculate_bmi(weight, height):
    try:
        weight = float(weight)
        height = float(height)
        if weight <= 0 or height <= 0:
            raise ValueError("Weight and height must be positive numbers.")
        bmi = weight / (height ** 2)
        return round(bmi, 2)
    except ValueError as e:
        return str(e)

# Function to classify BMI
def classify_bmi(bmi):
    if bmi < 18.5:
        return "Underweight"
    elif 18.5 <= bmi < 24.9:
        return "Normal weight"
    elif 25 <= bmi < 29.9:
        return "Overweight"
    else:
        return "Obesity"

# Function to store data in SQLite database
def store_data(weight, height, bmi, category):
    conn = sqlite3.connect('bmi_data.db')
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS bmi_records
                 (weight REAL, height REAL, bmi REAL, category TEXT)''')
    c.execute('INSERT INTO bmi_records (weight, height, bmi, category) VALUES (?, ?, ?, ?)',
              (weight, height, bmi, category))
    conn.commit()
    conn.close()

# Function to display BMI results
def show_result():
    weight = weight_entry.get()
    height = height_entry.get()
    bmi = calculate_bmi(weight, height)
    if isinstance(bmi, str):  # Error message
        messagebox.showerror("Error", bmi)
        return
    category = classify_bmi(bmi)
    result_label.config(text=f"Your BMI is {bmi}. You are classified as: {category}")
    store_data(weight, height, bmi, category)
    plot_bmi_data()

# Function to plot historical BMI data
def plot_bmi_data():
    conn = sqlite3.connect('bmi_data.db')
    c = conn.cursor()
    c.execute('SELECT weight, height, bmi FROM bmi_records')
    data = c.fetchall()
    conn.close()

    if not data:
        messagebox.showinfo("No Data", "No historical data to plot.")
        return

    weights, heights, bmis = zip(*data)

    plt.figure(figsize=(6, 4))
    plt.plot(bmis, marker='o', linestyle='-', color='b')
    plt.title('Historical BMI Data')
    plt.xlabel('Record')
    plt.ylabel('BMI')
    plt.grid(True)

    # Clear the previous plot if any
    for widget in plot_frame.winfo_children():
        widget.destroy()

    # Embed the plot in the Tkinter window
    canvas = FigureCanvasTkAgg(plt.gcf(), master=plot_frame)
    canvas.draw()
    canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)

# Setting up the Tkinter GUI
root = tk.Tk()
root.title("BMI Calculator")

# Input fields
tk.Label(root, text="Weight (kg):").pack()
weight_entry = tk.Entry(root)
weight_entry.pack()

tk.Label(root, text="Height (m):").pack()
height_entry = tk.Entry(root)
height_entry.pack()

# Calculate button
calculate_button = tk.Button(root, text="Calculate BMI", command=show_result)
calculate_button.pack()

# Result display
result_label = tk.Label(root, text="")
result_label.pack()

# Plotting frame
plot_frame = tk.Frame(root)
plot_frame.pack(fill=tk.BOTH, expand=True)

# Run the GUI
root.mainloop()